# Gallery of Large Galaxies

The purpose of this notebook is to build the gallery of large galaxies for the various Legacy Survey data releases.

The parent sample is a diameter-limited (*D25>5* arcsec) sample defined and documented as part of the [Legacy Survey Large Galaxy Atlas](https://github.com/moustakas/LSLGA).  

### Imports and paths

In [1]:
import os
import subprocess
import numpy as np

import fitsio
import matplotlib.pyplot as plt
from astropy.table import Table, vstack
from PIL import Image, ImageDraw, ImageFont

from astrometry.util.util import Tan
from astrometry.util.fits import fits_table, merge_tables
from legacypipe.survey import ccds_touching_wcs, LegacySurveyData

In [2]:
import multiprocessing
nproc = multiprocessing.cpu_count() // 2

In [3]:
%matplotlib inline

### Preliminaries

Define the data release and specify the D(25) angular diameter range (in arcmin) of the galaxies in the gallery.

In [4]:
d25min, d25max = 1.0, 5.0 # [arcmin]
dr = 'dr5'
if dr == 'dr5':
    PIXSCALE = 0.262
else:
    raise NotImplementedError

DIAMFACTOR = 4

In [5]:
gallerydir = os.path.join( os.getenv('SCRATCH'), dr, 'gallery' )
galleryfile = os.path.join(gallerydir, 'gallery-large-galaxies-{}.fits'.format(dr))

In [6]:
survey = LegacySurveyData()

### Define the parent sample based on cuts to D(25).

In [7]:
def _catalog_template(nobj=1):
    cols = [
        ('GALAXY', 'S28'), 
        ('PGC', 'S10'), 
        ('RA', 'f8'), 
        ('DEC', 'f8'),
        ('TYPE', 'S8'),
        ('MULTIPLE', 'S1'),
        ('RADIUS', 'f4'),
        ('BA', 'f4'),
        ('PA', 'f4'),
        ('BMAG', 'f4'),
        ('IMAG', 'f4'),
        ('VHELIO', 'f4'),
        ('BRICKNAME', 'S17', (4,))
        ]
    catalog = Table(np.zeros(nobj, dtype=cols))
    catalog['RADIUS'].unit = 'arcsec'
    catalog['VHELIO'].unit = 'km/s'

    return catalog

In [8]:
def read_leda(gallerydir='.', d25min=0.0, d25max=1000.0, decmin=-90.0, 
              decmax=+90.0, ramin=0.0, ramax=360.0):
    """Read the parent LEDA catalog and put it in a standard format.
    
    """
    ledafile = os.path.join(gallerydir, 'leda-logd25-0.05.fits')
    print('Reading {}'.format(ledafile))
    cat = Table.read(ledafile)

    outcat = _catalog_template(len(cat))
    outcat['GALAXY'] = cat['GALAXY']
    outcat['PGC'] = cat['PGC']
    outcat['RA'] = cat['RA']
    outcat['DEC'] = cat['DEC']
    outcat['TYPE'] = cat['TYPE']
    outcat['MULTIPLE'] = cat['MULTIPLE']
    outcat['RADIUS'] = cat['D25']/2.0 # semi-major axis radius [arcsec]
    outcat['BA'] = cat['BA']
    outcat['PA'] = cat['PA']
    outcat['BMAG'] = cat['BMAG']
    outcat['IMAG'] = cat['IMAG']
    outcat['VHELIO'] = cat['VHELIO']

    these = np.where((outcat['RADIUS']*2/60.0 <= d25max) *
                     (outcat['RADIUS']*2/60.0 >= d25min) *
                     (outcat['DEC'] <= decmax) *
                     (outcat['DEC'] >= decmin) *
                     (outcat['RA'] <= ramax) *
                     (outcat['RA'] >= ramin)
                     )[0]
    outcat = outcat[these] 
    print('Found {}/{} galaxies with D25 = {}-{} arcmin.'.format(
            len(outcat), len(cat), d25min, d25max))

    return outcat

In [9]:
bricks = survey.get_bricks()
allccds = survey.get_annotated_ccds()
cut = survey.photometric_ccds(allccds)
if cut is not None:
    allccds.cut(cut)
cut = survey.ccd_cuts(allccds)
allccds.cut(cut == 0)
print('Read {} CCDs.'.format(len(allccds)))

Converted brickname from |S8 to <U8
Reading annotated CCDs from /global/cscratch1/sd/desiproc/dr5/ccds-annotated-run19.fits.gz
Converted object from |S37 to <U37
Converted filter from |S1 to <U1
Converted date_obs from |S10 to <U10
Converted ut from |S15 to <U15
Converted ha from |S13 to <U13
Converted propid from |S10 to <U10
Converted ccdname from |S3 to <U3
Converted camera from |S5 to <U5
Converted expid from |S12 to <U12
Converted image_filename from |S51 to <U51
Converted plver from |S6 to <U6
Got 73860 CCDs
Reading annotated CCDs from /global/cscratch1/sd/desiproc/dr5/ccds-annotated-decals.fits.gz
Converted object from |S37 to <U37
Converted filter from |S1 to <U1
Converted date_obs from |S10 to <U10
Converted ut from |S15 to <U15
Converted ha from |S13 to <U13
Converted propid from |S10 to <U10
Converted ccdname from |S3 to <U3
Converted camera from |S5 to <U5
Converted expid from |S12 to <U12
Converted image_filename from |S61 to <U61
Converted plver from |S6 to <U6
Got 493440

In [10]:
parent = read_leda(gallerydir=gallerydir, d25min=d25min, d25max=d25max)

Reading /global/cscratch1/sd/ioannis/dr5/gallery/leda-logd25-0.05.fits
Found 22385/2143628 galaxies with D25 = 1.0-5.0 arcmin.


### Build the parent sample of galaxies in the DRX footprint

In [11]:
def _uniqccds(ccds):
    '''Get the unique set of CCD files.'''
    ccdfile = []
    [ccdfile.append('{}-{}'.format(expnum, ccdname)) for expnum,
     ccdname in zip(ccds.expnum, ccds.ccdname)]
    _, indx = np.unique(ccdfile, return_index=True)
    return ccds[indx]

In [12]:
def _galwcs(gal, factor=DIAMFACTOR):
    '''Build a simple WCS object for a single galaxy.
    
    '''
    diam = factor*np.ceil(2.0*gal['RADIUS']/PIXSCALE).astype('int16') # [pixels]
    galwcs = Tan(gal['RA'], gal['DEC'], diam/2+0.5, diam/2+0.5,
                 -PIXSCALE/3600.0, 0.0, 0.0, PIXSCALE/3600.0, 
                 float(diam), float(diam))
    return galwcs

In [13]:
def _build_sample_onegalaxy(args):
    """Filler function for the multiprocessing."""
    return build_sample_onegalaxy(*args)

In [14]:
def build_sample_onegalaxy(gal, allccds, ccdsdir, bricks, survey):
    """Wrapper function to find overlapping CCDs for a given galaxy.

    First generously find the nearest set of CCDs that are near the galaxy and
    then demand that there's 3-band coverage in a much smaller region centered
    on the galaxy.

    """
    #print('Working on {}...'.format(gal['GALAXY'].strip()))
    galwcs = _galwcs(gal)
    these = ccds_touching_wcs(galwcs, allccds)

    if len(these) > 0:
        ccds1 = _uniqccds( allccds[these] )

        # Is there 3-band coverage?
        galwcs_small = _galwcs(gal, factor=0.5)
        these_small = ccds_touching_wcs(galwcs_small, ccds1)
        ccds1_small = _uniqccds( ccds1[these_small] )

        if 'g' in ccds1_small.filter and 'r' in ccds1_small.filter and 'z' in ccds1_small.filter:
            print('For {} found {} CCDs, RA = {:.5f}, Dec = {:.5f}, Radius={:.4f} arcsec'.format(
                gal['GALAXY'].strip(), len(ccds1), gal['RA'], gal['DEC'], gal['RADIUS']))

            ccdsfile = os.path.join(ccdsdir, '{}-ccds.fits'.format(gal['GALAXY'].strip().lower()))
            #print('  Writing {}'.format(ccdsfile))
            #if os.path.isfile(ccdsfile):
            #    os.remove(ccdsfile)
            #ccds1.writeto(ccdsfile)

            # Also get the set of bricks touching this footprint.
            rad = 2*gal['RADIUS']/3600 # [degree]
            brickindx = survey.bricks_touching_radec_box(bricks,
                                                         gal['RA']-rad, gal['RA']+rad,
                                                         gal['DEC']-rad, gal['DEC']+rad)
            if len(brickindx) == 0 or len(brickindx) > 4:
                print('This should not happen!')
                pdb.set_trace()
            gal['BRICKNAME'][:len(brickindx)] = bricks.brickname[brickindx]

            return [gal, ccds1]

    return None

In [15]:
ccdsdir = os.path.join(gallerydir, 'ccds')

sampleargs = list()
for cc in parent:
    sampleargs.append( (cc, allccds, ccdsdir, bricks, survey) )

nproc = 1
if nproc > 1:
    p = multiprocessing.Pool(nproc)
    result = p.map(_build_sample_onegalaxy, sampleargs)
    p.close()
else:
    result = list()
    for args in sampleargs:
        result.append(_build_sample_onegalaxy(args))

# Remove non-matching galaxies and write out the sample
result = list(filter(None, result))
result = list(zip(*result))

outcat = vstack(result[0])
outccds = merge_tables(result[1])

if os.path.isfile(galleryfile):
    os.remove(galleryfile)
            
print('Writing {}'.format(galleryfile))
outcat.write(galleryfile)

For UGC12890 found 10 CCDs, RA = 0.02925, Dec = 8.27911, Radius=46.4645 arcsec
For PGC000012 found 18 CCDs, RA = 0.03600, Dec = -6.37392, Radius=42.3761 arcsec
For PGC000023 found 32 CCDs, RA = 0.08955, Dec = -2.61203, Radius=43.3632 arcsec
For UGC12913 found 19 CCDs, RA = 0.40290, Dec = 3.50558, Radius=34.4446 arcsec
For PGC000176 found 16 CCDs, RA = 0.64515, Dec = -3.71077, Radius=32.1456 arcsec
For PGC000192 found 25 CCDs, RA = 0.70275, Dec = -3.60606, Radius=32.8943 arcsec
For UGC00005 found 31 CCDs, RA = 0.77355, Dec = -1.91384, Radius=41.4115 arcsec
For UGC00009 found 7 CCDs, RA = 0.82545, Dec = 4.62750, Radius=30.0000 arcsec
For UGC00015 found 10 CCDs, RA = 0.93675, Dec = 4.29811, Radius=34.4446 arcsec
For UGC00033 found 3 CCDs, RA = 1.24080, Dec = 5.12347, Radius=36.0679 arcsec
For PGC000312 found 14 CCDs, RA = 1.27230, Dec = -7.09339, Radius=64.1389 arcsec
For NGC7827 found 3 CCDs, RA = 1.36530, Dec = 5.22233, Radius=38.6475 arcsec
For NGC7832 found 19 CCDs, RA = 1.61850, Dec 

### Retrieve FITS cutouts of each galaxy in each band

In [16]:
fitsdir = os.path.join(gallerydir, 'fits')
if not os.path.isdir(fitsdir):
    os.mkdir(fitsdir)

In [17]:
sample = Table.read(galleryfile)
sample

GALAXY,PGC,RA,DEC,TYPE,MULTIPLE,RADIUS,BA,PA,BMAG,IMAG,VHELIO,BRICKNAME [4]
str28,str10,float64,float64,str4,str1,float32,float32,float32,float32,float32,float32,str20
UGC12890,PGC0000014,0.02925,8.27911,E,,46.4645,0.537032,9.5,15.78,-999.0,11602.0,0001p082 ..
PGC000012,PGC0000012,0.036,-6.37392,Sa,,42.3761,0.218776,168.2,14.84,-999.0,6546.0,0001m065 ..
PGC000023,PGC0000023,0.08955,-2.61203,E-S0,,43.3632,0.512861,127.5,15.17,12.71,11365.0,0001m027 ..
UGC12913,PGC0000124,0.4029,3.50558,Sc,,34.4446,0.144544,4.3,16.3,14.61,6339.0,0003p035 ..
PGC000176,PGC0000176,0.64515,-3.71077,Sbc,,32.1456,0.616595,179.8,14.49,12.84,6465.0,0006m037 ..
PGC000192,PGC0000192,0.70275,-3.60606,Sc,,32.8943,0.269153,25.0,15.24,-999.0,6212.0,0006m035 ..
UGC00005,PGC0000205,0.77355,-1.91384,Sbc,,41.4115,0.457088,47.0,14.2,12.17,7297.0,0008m020 ..
UGC00009,PGC0000228,0.82545,4.6275,,,30.0,0.660693,118.0,16.32,-999.0,29568.0,0008p045 ..
UGC00015,PGC0000258,0.93675,4.29811,Sab,,34.4446,0.251189,63.6,16.07,-999.0,11577.0,0008p042 ..


In [28]:
for gal in sample:
    galaxy = gal['GALAXY'].strip().lower()

    size = DIAMFACTOR*np.ceil(gal['RADIUS']/PIXSCALE).astype('int16') # [pixels]

    # Get a FITS cutout and then split the file into three individual bands.
    baseurl = 'http://legacysurvey.org/viewer-dev/fits-cutout/'
    fitsurl = '{}?ra={:.6f}&dec={:.6f}&pixscale={:.3f}&size={:g}&layer=decals-{}'.format(
        baseurl, gal['RA'], gal['DEC'], PIXSCALE, size, dr)
    fitsfile = os.path.join(fitsdir, '{}.fits'.format(galaxy))
    cmd = 'wget --continue -O {:s} "{:s}"' .format(fitsfile, fitsurl)
    if os.path.isfile(os.path.join(fitsdir, '{}-{}.fits'.format(galaxy, 'g'))):
        print('Skipping galaxy {}'.format(galaxy))
    else:
        print(cmd)
        os.system(cmd)

        img, hdr = fitsio.read(fitsfile, ext=0, header=True)
        for ii, band in enumerate(['g', 'r', 'z']):
            bandfile = os.path.join(fitsdir, '{}-{}.fits'.format(galaxy, band))
            print('  Writing {}'.format(bandfile))
            fitsio.write(bandfile, img[ii, :, :], clobber=True, header=hdr)        
        print('  Removing {}'.format(fitsfile))
        os.remove(fitsfile)

Skipping galaxy ugc12890
Skipping galaxy pgc000012
Skipping galaxy pgc000023
Skipping galaxy ugc12913
Skipping galaxy pgc000176
Skipping galaxy pgc000192
Skipping galaxy ugc00005
Skipping galaxy ugc00009
Skipping galaxy ugc00015
Skipping galaxy ugc00033
Skipping galaxy pgc000312
Skipping galaxy ngc7827
Skipping galaxy ngc7832
Skipping galaxy pgc000538
Skipping galaxy ugc00066
Skipping galaxy pgc000637
Skipping galaxy ngc0012
Skipping galaxy ugc00081
Skipping galaxy ugc00088
Skipping galaxy ugc00099
wget --continue -O /global/cscratch1/sd/ioannis/dr5/gallery/fits/ngc0038.fits "http://legacysurvey.org/viewer-dev/fits-cutout/?ra=2.945700&dec=-5.586280&pixscale=0.262&size=516&layer=decals-dr5"
  Writing /global/cscratch1/sd/ioannis/dr5/gallery/fits/ngc0038-g.fits
  Writing /global/cscratch1/sd/ioannis/dr5/gallery/fits/ngc0038-r.fits
  Writing /global/cscratch1/sd/ioannis/dr5/gallery/fits/ngc0038-z.fits
  Removing /global/cscratch1/sd/ioannis/dr5/gallery/fits/ngc0038.fits
wget --continue -O

TypeError: 'NoneType' object is not subscriptable

### Build a color image using trilogy.py

After the png image has been generated, read it back in and add a scale bar and a galaxy label.

In [29]:
pngdir = os.path.join(gallerydir, 'png')
if not os.path.isdir(pngdir):
    os.mkdir(pngdir)

In [30]:
barlen30 = np.round(30.0 / PIXSCALE).astype('int')
fonttype = os.path.join(gallerydir, 'Georgia.ttf')

In [31]:
for gal in sample:
    galaxy = gal['GALAXY'].strip().lower()
    pngfile = os.path.join(pngdir, '{}.png'.format(galaxy))

    paramfile = os.path.join(pngdir, '{}.in'.format(galaxy))
    with open(paramfile, 'w') as param:
        param.write('B\n')
        param.write(os.path.join(fitsdir, '{}-g.fits\n'.format(galaxy)))
        param.write('\n')
        param.write('G\n')
        param.write(os.path.join(fitsdir, '{}-r.fits\n'.format(galaxy)))
        param.write('\n')        
        param.write('R\n')
        param.write(os.path.join(fitsdir, '{}-z.fits\n'.format(galaxy)))
        param.write('\n')
        param.write('indir {}\n'.format(fitsdir))
        param.write('outname {}\n'.format(pngfile))
        param.write('noiselum 0.3\n')
        param.write('satpercent 0.001\n')
        param.write('samplesize 500\n')
        param.write('stampsize 5000\n')
        #param.write('scaling {}\n'.format(os.path.join(gallerydir, 'levels.txt')))
        param.write('colorsatfac 0.9\n')
        param.write('deletetests 1\n')
        param.write('showstamps 0\n')
        param.write('show 0\n')
        param.write('legend 0\n')
        param.write('testfirst 0\n')
        
    trilogy = os.path.join(os.getenv('LEGACYPIPE_DIR'), 'py', 'legacyanalysis', 'trilogy.py')
    cmd = 'python {} {}'.format(trilogy, paramfile)
    #print(cmd)
    os.system(cmd)
    for txt in ('levels.txt', 'trilogyfilterlog.txt'):
        thisfile = os.path.join(os.getenv('LEGACYPIPE_DIR'), 'doc', 'nb', txt)
        os.remove(thisfile)
    os.remove(os.path.join(pngdir, '{}_filters.txt'.format(galaxy)))
    
    im = Image.open(pngfile)
    sz = im.size
    fntsize = np.round(sz[0]/35).astype('int')
    width = np.round(sz[0]/175).astype('int')
    font = ImageFont.truetype(fonttype, size=fntsize)
    draw = ImageDraw.Draw(im)
    # Label the galaxy name--
    draw.text((0+fntsize*2, 0+fntsize*2), galaxy.upper(), font=font)
    #draw.text((sz[0]-fntsize*6, sz[1]-fntsize*3), galaxy.upper(), font=font)
    # Add a 30 arcsec scale bar--
    x0, x1, yy = sz[1]-fntsize*2-barlen30, sz[1]-fntsize*2, sz[0]-fntsize*2
    draw.line((x0, yy, x1, yy), fill='white', width=width)
    im.save(pngfile)    
    print('Wrote {}'.format(pngfile))

Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/ugc12890.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/pgc000012.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/pgc000023.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/ugc12913.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/pgc000176.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/pgc000192.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/ugc00005.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/ugc00009.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/ugc00015.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/ugc00033.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/pgc000312.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/ngc7827.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/ngc7832.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/pgc000538.png
Wrote /global/cscratch1/sd/ioannis/dr5/gallery/png/ugc00066.png
Wrote /global/cscratch1/sd/ioannis/d

FileNotFoundError: [Errno 2] No such file or directory: '/global/cscratch1/sd/ioannis/repos/legacypipe/doc/nb/levels.txt'

### Finally, assemble the webpage of good and rejected gallery images.

In [32]:
reject = ['ngc0520', 'sdssj100819.08+120322.4', 'ugc02302', 'ugc03974',
          'ngc3003', 'ngc3044', 'ugc05364', 'ngc3379', 'ngc3501']

In [33]:
sample = Table.read(galleryfile)
htmlfile = os.path.join(gallerydir, 'index.html')
htmlfile_reject = os.path.join(gallerydir, 'index-reject.html')
baseurl = 'http://legacysurvey.org/viewer-dev'

In [40]:
with open(htmlfile, 'w') as html:
    html.write('<html><body>\n')
    for gal in sample[:1830]:
        galaxy = gal['GALAXY'].strip().lower()
        if galaxy not in reject:
            pngfile = os.path.join('png', '{}.png'.format(galaxy))
            img = 'src="{}" alt="{}" width="500px"'.format(pngfile, galaxy.upper())
            html.write('<a href="{}/?ra={:.8f}&dec={:.8f}&layer=decals-{}"><img {}></a>\n'.format(
                    baseurl, gal['RA'], gal['DEC'], dr, img))
    html.write('</body></html>\n')

In [41]:
with open(htmlfile_reject, 'w') as html:
    html.write('<html><body>\n')
    for gal in sample[:1830]:
        galaxy = gal['GALAXY'].strip().lower()
        if galaxy in reject:
            pngfile = os.path.join('png', '{}.png'.format(galaxy))
            img = 'src="{}" alt="{}" width="500px"'.format(pngfile, galaxy.upper())
            html.write('<a href="{}/?ra={:.8f}&dec={:.8f}&layer=decals-{}"><img {}></a>\n'.format(
                    baseurl, gal['RA'], gal['DEC'], dr, img))
    html.write('</body></html>\n')